## **Linear Regression**
This notebook presents the issue of linear regression along with the explanation of individual sections, enabling an in-depth analysis of this issue. This course is based on aakashns's PyTorch for Deep Learning tutorial [linear-regression](https://jovian.ai/aakashns/02-linear-regression). </br>
This notebook will be very similar to one from tutorial. Purpose of it is to help learn linear regression instead of making different notebook. 
However, some could things will be slightly different.




First we need to install the required libraries. The installation of **PyTorch** may differ based on your operating system or hardware.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder / Colab
!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html


Next step is to **inport** module. In addition to ***PyTorch*** we will also need ***numpy***.

Numpy is essential because it provides:
*   **Autograd**: The ability to automatically compute gradients for tensor operations is essential for training deep learning models.
*   **GPU support**: While working with massive datasets and large models, PyTorch tensor operations can be performed efficiently using a Graphics Processing Unit (GPU). Computations that might typically take hours can be completed within minutes using GPUs.






In [ ]:
import torch
import numpy as np

## Linear Regression

*Linear regression* is one of the foundational algorithms in machine learning. In this section we will be creating a model that predicts crop yields for apples and oranges (***target variables***). </br>
To achive this we will be computing the average temperature, rainfall, and humidity which are (***input variables or features***). 

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

Linear regression model's target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

Now we need to load data. </br> We have to load verage temperature, rainfall, and humidity (in this order) as ***input variables***. </br> Then we will load crops as ***targets***. </br>
Also we are specyfic type of data, in this case is floating point number that is occupying 32 bits in computer memory.

In [ ]:
# Input variables -> in order temperature, rainfall, humidity.
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets -> crops in order apples, oranges.
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Now we are converting arrays to tensors. </br>
This is very common aproach since most of data will be in CSV format. </br>
It's very easy, we just need to use PyTorch method calles ```torch.from_numpy()``` where we just need to give numpy array.

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

At the begginig values of weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices. </br>
We will initialized them as random values (and later we will try to find values that are closer to target). </br> The first row of `w` and the first element of `b` are used to predict the first target variable (in this case apples), and similarly, the second for oranges. </br> </br>

We will be using `torch.randn()` that creates a tensor with the given shape, with elements picked randomly from a normal distribution with ***mean = 0*** and ***standard deviation 1***.

In [ ]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

Our ***model*** is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png) </br> </br>
`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

In [ ]:
# Model: 
def model(x):
    return x @ w.t() + b

Now we can predict values using our model and compare them with values from table.

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[173.1329,  20.7676],
        [232.4916,  39.6536],
        [259.9453,  30.7304],
        [170.9321,   1.7364],
        [227.4406,  56.6079]], grad_fn=<AddBackward0>)


Now we will show the actual targets.

In [ ]:
# actual targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


There is no similarity and shouldn't be because our model is initialized with random vlaues. </br>
To improve it we need to we need a way to *evaluate how well our model is performing*. </br> We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [ ]:
# MSE loss function
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()


*   `torch.sum` returns the sum of all the elements in a tensor. 
*   `.numel` method of a tensor returns the number of elements in a tensor.



In [ ]:
# Compute and show loss at this point (model with random values)
# Model is better when loss value is as small as possible
loss = mse(preds, targets)
print(loss)

tensor(11598.4160, grad_fn=<DivBackward0>)


With PyTorch, we can automatically compute the gradient (indicating the directions of the fastest increases in the value of a given scalar field) or derivative of the loss to the weights and biases because they have `requires_grad` set to `True`. 

The gradients are stored in the .grad property of the respective tensors.

In [ ]:
# Compute gradients
loss.backward()

Step above is very important. We need to ensure to compute gradients

In [ ]:
# Gradients for weights compared to weights
print(w)
print(w.grad)

tensor([[ 0.8724,  0.8349,  1.2439],
        [-0.4301, -0.0584,  1.3264]], requires_grad=True)
tensor([[11675.2285, 11683.2393,  7425.6650],
        [-5195.1753, -5981.4297, -3529.3989]])


## Adjusting weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss with reference to any individual weight or bias element, it will look like the figure shown below.  </br> An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) with reference to the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss with reference to that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

# Coding it
We can subtract from each weight element a small quantity proportional to the derivative of the loss with reference to that element to reduce the loss slightly.

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. This number is called the ***learning rate*** of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [ ]:
# Applying reducing of loss function
loss = mse(preds, targets)
print(loss)

tensor(11598.4160, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. </br> **This is requaired because PyTorch accumulates gradients.** </br>Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Training the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[153.5878,  30.0859],
        [206.8321,  51.9043],
        [229.8240,  45.3130],
        [151.2507,  10.9140],
        [202.9694,  68.4059]], grad_fn=<AddBackward0>)


In [ ]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(7927.1431, grad_fn=<DivBackward0>)


In [ ]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 9659.5547,  9523.3721,  6091.4902],
        [-4232.3926, -4944.6895, -2890.2083]])
tensor([112.6928, -50.6754])


In [ ]:
# Now update the weights and biases using the gradients computed above.
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
# Let's take a look at the new weights and biases
print(w)
print(b)

tensor([[ 0.6591,  0.6229,  1.1087],
        [-0.3359,  0.0509,  1.3906]], requires_grad=True)
tensor([ 0.0157, -0.9552], requires_grad=True)


Now With the new weights and biases, the model should have a lower loss

In [ ]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5451.8154, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an **_epoch_**.</br> Let's train the model for 100 epochs.

In [ ]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
# Calculate loss after 100 epochs
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(111.3868, grad_fn=<DivBackward0>)


As we can see loss in now much lower then at the beggining. </br>
Lets compare predictions and targets:



In [ ]:
# Predictions
preds

tensor([[ 60.8015,  71.3727],
        [ 86.5965, 106.5045],
        [102.8638, 117.9762],
        [ 42.4033,  44.6409],
        [ 97.1159, 124.6495]], grad_fn=<AddBackward0>)

In [ ]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear Regression with build in PyTorch


In [ ]:
import torch.nn as nn

As before, we are loading all nessesery data

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [ ]:
# import functionality
from torch.utils.data import TensorDataset

In [ ]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.
</br></br>
We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [ ]:
from torch.utils.data import DataLoader
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a for loop. Let's look at an example.

In [ ]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 88., 134.,  59.],
        [ 74.,  66.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 135.,  57.],
        [ 87., 134.,  58.]])
tensor([[118., 132.],
        [ 57.,  69.],
        [ 81., 101.],
        [118., 134.],
        [119., 133.]])


In each iteration, the data loader returns one batch of data with the given batch size. </br>If `shuffle` is set to `True`, it shuffles the training data before creating batches. </br>*Shuffling helps randomize the input to the optimization algorithm, leading to a* ***faster reduction in the loss***.
</br> </br>
## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [ ]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5124,  0.1387,  0.1039],
        [-0.2434, -0.1633,  0.1130]], requires_grad=True)
Parameter containing:
tensor([ 0.2457, -0.5621], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [ ]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.5124,  0.1387,  0.1039],
         [-0.2434, -0.1633,  0.1130]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2457, -0.5621], requires_grad=True)]

In [ ]:
# Generate predictions
# We can use model just as previously
preds = model(inputs)
preds

tensor([[-23.3995, -24.4121],
        [-27.5283, -29.8494],
        [-19.7238, -37.0634],
        [-42.2098, -28.2316],
        [-14.5232, -25.1219],
        [-24.0505, -24.4923],
        [-27.5630, -29.5731],
        [-20.1323, -37.1938],
        [-41.5588, -28.1514],
        [-13.9069, -24.7654],
        [-23.4342, -24.1359],
        [-28.1793, -29.9296],
        [-19.6890, -37.3396],
        [-42.8261, -28.5880],
        [-13.8721, -25.0417]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [ ]:
# Import nn.functional
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

# Compute the loss for the current predictions of our model.
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13501.3926, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. </br> SGD is short for "stochastic gradient descent". The term **_stochastic_ indicates** that samples are selected in random batches instead of as a single group.

In [ ]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs and compare resoults.

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

# Generate predictions
preds = model(inputs)
preds

Epoch [10/100], Loss: 143.0284
Epoch [20/100], Loss: 172.9396
Epoch [30/100], Loss: 84.4990
Epoch [40/100], Loss: 73.8386
Epoch [50/100], Loss: 31.1499
Epoch [60/100], Loss: 53.3417
Epoch [70/100], Loss: 40.2082
Epoch [80/100], Loss: 21.7125
Epoch [90/100], Loss: 12.9748
Epoch [100/100], Loss: 7.0584


tensor([[ 57.5894,  71.2395],
        [ 81.0094,  99.2403],
        [119.2657, 133.8689],
        [ 24.3217,  43.4286],
        [ 98.1019, 112.8588],
        [ 56.3990,  70.2701],
        [ 80.6553,  99.0531],
        [119.4640, 134.3583],
        [ 25.5121,  44.3979],
        [ 98.9382, 113.6409],
        [ 57.2352,  71.0523],
        [ 79.8190,  98.2710],
        [119.6198, 134.0560],
        [ 23.4854,  42.6464],
        [ 99.2923, 113.8281]], grad_fn=<AddmmBackward>)

In [ ]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

The final result is very good. </br> The loss value is just 7. Values of predictions are similar to the targets.